# Prompting an LLM in code

**Code cell 1:** Set up programming environment to use code to send prompts to OpenAI's cloud-hosted service. 

## 简单版

In [9]:
import os
import openai

# 获取api_key
openai.api_key = os.getenv('OPENAI_API_KEY')

In [8]:
# 判断是否设置了 API 密钥
if openai.api_key is None:
    print("环境变量 'OPENAI_API_KEY' 未设置。")
else:
    print("环境变量 'OPENAI_API_KEY' 已设置。")
    print("API 密钥: " + openai.api_key)  # 注意：出于安全考虑，通常不建议打印API密钥

环境变量 'OPENAI_API_KEY' 已设置。
API 密钥: sk-proj-O6jZufbDxTGDyQgZMtcdT3BlbkFJSknNahfWg4F7kbp8z33A


In [5]:
# 调用gpt model的函数
client = openai.OpenAI()

def llm_response(prompt, model="gpt-4"):  # gpt-3.5-turbo
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

**Code cell 2:** Define a prompt that will classify the sentiment of a restaurant review. 

In [7]:
prompt = '''
    Classify the following reviews 
    as having either a positive or
    negative sentiment:

    1.The banana pudding was really tasty!
    2.The banana pudding was really bad!
'''

response = llm_response(prompt)
print(response)

1. Positive
2. Negative


## 高阶版--总代码

In [1]:
import os
import openai

# 获取环境变量值
openai.api_key = os.getenv('OPENAI_API_KEY')

# 调用gpt model的函数
client = openai.OpenAI()
def llm_response(prompt, model="gpt-4"):  # gpt-3.5-turbo
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

# 主功能函数
def analyze_reviews(file_path):
    # 读取文件中的所有评论
    try:
        with open(file_path, 'r', encoding='utf-8') as file: 
            # 打开一个位于 file_path 的文件，'r' 表示以只读模式打开文件，encoding='utf-8' 指定文件的编码方式为UTF-8
            # 通过 with 语句，Python 创建了一个上下文环境，使得在这个环境中打开的文件在代码块执行完毕后自动关闭
            reviews = file.read().strip().split('\n')  # 确保文件末尾无空行
    except FileNotFoundError:
        print("Error: File not found.")
        return

    # 分析每条评论的情感并准备保存结果
    results = []
    for review in reviews:
        prompt = f"Classify the sentiment of the following review: {review}"
        sentiment = llm_response(prompt)
        results.append(f"Review: {review} - Sentiment: {sentiment}")

    # 将结果保存到文件
    try:
        with open('sentiment_analysis_results.txt', 'w', encoding='utf-8') as output_file:
            for result in results:
                output_file.write(result + "\n")
        print("Analysis results saved to 'sentiment_analysis_results.txt'.")
    except Exception as e:
        print(f"Error writing to file: {e}")

# 用来控制只有在直接运行脚本时才执行某些代码（而不是作为模块被其他脚本导入时）
if __name__ == "__main__":
    analyze_reviews('reviews.txt')

Analysis results saved to 'sentiment_analysis_results.txt'.
